Reference = https://medium.com/daangn/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9C%BC%EB%A1%9C-%EB%8F%99%EB%84%A4%EC%83%9D%ED%99%9C-%EA%B2%8C%EC%8B%9C%EA%B8%80-%ED%95%84%ED%84%B0%EB%A7%81%ED%95%98%EA%B8%B0-263cfe4bc58d

In [1]:
import pandas as pd
import pickle

# Process
 ### BERT Pretrain
 - 스팸 분류를 위해서 기존 유튜브의 일반적인 댓글들에 대해 pretrain이 필요  
 -- Youtube 댓글 크롤링 (완료)  
 -- 댓글 전처리 & 토크나이즈 (wordpiece or sentencepiece or mecab이용한 bert tokenize-electra)  
 -- 학습을 위해선 문장 2개를 '\t'로 붙이고, 마지막에 '\n'으로 데이터를 구성 (?)  
 -- 학습
 ### Dataset 구축
 - 필터링을 위한 데이터 구축
 -- 당근마켓의 경우 실제 필요한 게시글 7290 / 필요 없는 게시글 710 라벨링  
 -- 8000개 학습 데이터 중 2000개를 테스트 데이터로 활용  
 -- sklearn을 통해 지표확인(recall 높게 유지하면서 precision 높이는 것 - 맞는 게시글을 미노출 시키는 경우 방지)
 

## 스팸 데이터셋 구축

In [2]:
import os

path_dir = './'
file_list = os.listdir(path_dir)

In [3]:
#크롤링된 데이터 불러오기
df_all = pd.DataFrame()
for file in file_list:
    if 'trending_video' in file:
        temp_df = pd.read_pickle(file)
        df_all = pd.concat([df_all, temp_df])
        
#중복, 결측치 제거
df_all.dropna(inplace=True)
df_all.drop_duplicates(inplace=True)

In [4]:
review_list = list(df_all['review'])

In [6]:
len(review_list)

49672

In [51]:
def get_KeywordDF(keyword):
    temp_list = []    
    for review in review_list:
        if keyword in review:
            temp_list.append(review)
            
    for idx, review in enumerate(temp_list):
        print('인덱스: {}'.format(idx))
        print('\n')
        print(review)
        print('--------')

    return list(set(temp_list))

In [8]:
spam_list = []

### 키워드로 스팸 찾기
- 댓글을 보다보면 일관된 단어들이 보인다.  
- 하나의 키워드로 필터링을 하고, 필터링 된 스팸에서 다른 키워드를 찾아 이전의 키워드에서 찾아내지 못한 스팸을 찾아낸다.

In [14]:
spam_1 = get_KeywordDF('채널')
spam_1

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '#이찬원 #유튜브채널\n2020년 한해는 찬또배기 이찬원님  저에게 주는 기쁨과 즐거움 행복함  넘 고맙고  감사합니다~항상 찬스들이 곁에서 응원합니다',
 '진정성이 너무 무시되면 저질채널이 될수있습니다 좀더 신경써주셍_',
 '꼰대희채널은 밥묵자가진리!\n잘보갑니다!',
 '원장님도 asmr 채널 만들어주세여 ㅠㅠ 목소리도 연기력도 대박\U0001f97a',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '권재관씨 개그 채널 도전하세요~\n구독하고 보고 싶습니다~\n어려운 시기에 개그보고 \n힘든일 잊고 싶고 힘내게 되요~\n진짜 재밌었는데',
 '찬원님  유튜버 채널  개설  진심으로 추카추카   합니다   영웅님팬으로써  형이  사랑하는 찬원님  앞으로  구독자가  빠르게  빠르게 늘어나도록 진심으로 바라겠어요  저도 구독했어요 앞으로도 자주 들려서  찬원님 소식 듣겠습니다   항상 건강잘   챙기시구요',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급

### 채널명으로 필터링
- 대체적으로채널의 주인에 대한 호칭(채널명, 혹은 별명)이 포함된 댓글은 스팸이 아닌 경우가 많다.  
- 대신 몇몇 스팸들의 경우, 

In [18]:
temp_spam = []
for spam in spam_1:
    if '이찬원' in spam:
        pass
    elif '찬원' in spam:
        pass
    elif '원숙' in spam:
        pass
    elif '꼰대희' in spam:
        pass
    elif '대희' in spam:
        pass
    elif '권재관' in spam:
        pass
    elif '재관' in spam:
        pass
    elif '희극인' in spam:
        pass
    elif '감스트' in spam:
        pass
    elif '다정한 부부' in spam:
        pass
    elif '개그맨' in spam:
        pass
    elif 'RC' in spam:
        pass
    elif 'F5' in spam:
        pass
    elif '육아대디' in spam:
        pass
    elif '토모' in spam:
        pass
    elif '올리버' in spam:
        pass
    else:
        temp_spam.append(spam)

In [19]:
temp_spam

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '진정성이 너무 무시되면 저질채널이 될수있습니다 좀더 신경써주셍_',
 '원장님도 asmr 채널 만들어주세여 ㅠㅠ 목소리도 연기력도 대박\U0001f97a',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다',
 '내 채널에 상해람보 있음 한번 봐보셈',
 '@이벤트알려주는채널 ㅈㄹ ㄴㄴ',
 '엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..',
 '@이벤트알려주는채널 그만좀해라 넌',
 '지 채널 홍보를 왜 여기서해',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지? 난독증인가? 좋아요를 눌러달라거나 문상있으니 자기 채널에 들어오라는 말도 없구만',
 '진짜 게스트가 꼰채널 살리네ㅎ',
 '제 채널에 엄청난 보물이 있어요!!',
 

### 리스트 삭제
- 하나하나 삭제하면 인덱스를 찾기가 어렵다..  
- 하나 삭제하고 나면 인덱스가 바뀌니 인덱스와 출력을 같이 해주는 함수가 필요

In [11]:
def del_and_prt(li, idx):
    del li[idx]
    for idx, review in enumerate(li):
        print('인덱스: {}'.format(idx))
        print('\n')
        print(review)
        print('--------')
    return li

In [25]:
a = del_and_prt(temp_spam, 1)
a

인덱스: 0


학생,주부들 부업거리로👍👍👍
하루30분만 광고봐도 1만원
제 채널 영상확인하세요
--------
인덱스: 1


@하루1만원씩 챙겨가세요 채널클릭 관종2
--------
인덱스: 2


제 채널에 고추 길어지는법이 있습니다!~ 😘😘
--------
인덱스: 3


뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅
--------
인덱스: 4


제 채널에 오셔서 2만원 받아가세요~ 
1주일 남음 서두르세요!
(12월 31일까지)
--------
인덱스: 5


언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖
--------
인덱스: 6


@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요
--------
인덱스: 7


님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여
--------
인덱스: 8


제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?
--------
인덱스: 9


근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다
--------
인덱스: 10


내 채널에 상해람보 있음 한번 봐보셈
--------
인덱스: 11


@이벤트알려주는채널 ㅈㄹ ㄴㄴ
--------
인덱스: 12


엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..
--------
인덱스: 13


@이벤트알려주는채널 그만좀해라 넌
--------
인덱스: 14


지 채널 홍보를 왜 여기서해
--------
인덱스: 15


제 채널들어와봐용

공짜(돈/캐시/깊티)받아가세요
피해 ㄴㄴㄴ!
--------
인덱스: 16


@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다',
 '내 채널에 상해람보 있음 한번 봐보셈',
 '@이벤트알려주는채널 ㅈㄹ ㄴㄴ',
 '엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..',
 '@이벤트알려주는채널 그만좀해라 넌',
 '지 채널 홍보를 왜 여기서해',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지? 난독증인가? 좋아요를 눌러달라거나 문상있으니 자기 채널에 들어오라는 말도 없구만',
 '진짜 게스트가 꼰채널 살리네ㅎ',
 '제 채널에 엄청난 보물이 있어요!!',
 '@바두자 님이나 하고 이런 채널 와서 그런짓 하지 마세요',
 '와 탑현님 채널에도 댓썼는데...',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '@mmm제채널 하루1만원 받아가세요mmm 조까',
 '역대급 bj 방송사

In [304]:
a = del_and_prt(a, 153)
a

인덱스: 0


학생,주부들 부업거리로👍👍👍
하루30분만 광고봐도 1만원
제 채널 영상확인하세요
--------
인덱스: 1


제 채널에 고추 길어지는법이 있습니다!~ 😘😘
--------
인덱스: 2


제 채널에 오셔서 2만원 받아가세요~ 
1주일 남음 서두르세요!
(12월 31일까지)
--------
인덱스: 3


제 채널들어와봐용

공짜(돈/캐시/깊티)받아가세요
피해 ㄴㄴㄴ!
--------
인덱스: 4


@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!
--------
인덱스: 5


역대급 bj 방송사고 터진듯?
걸그룹 연습생 출신이라더라
제 채널 와서 보세요..
--------
인덱스: 6


제 채널 들어와봐용 

공짜(돈/캐시/깊티) 받아가세요 !
사기 ㄴㄴ
--------
인덱스: 7


제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.
--------
인덱스: 8


제 채널에 좋은거 있어요
--------
인덱스: 9


제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤
--------
인덱스: 10


제 채널 들어와봐용 

공짜(캐시/돈/깊티) 받아가세용
피해 ㄴㄴ
--------
인덱스: 11


제 채널에 보면 큰일나는 영상있음
--------
인덱스: 12


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ
--------
인덱스: 13


제 채널에서 꽁돈 받아가세요🥳🥳🥳🥳
--------
인덱스: 14


제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!
절 믿고 따라오실수 있겠습니까?
(친구가 많다면 돈을 더 벌수도 있답니다!)
--------
인덱스: 15


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㄹㅁ
--------
인덱스: 16


제 채널에서 만원 받아가세
--------
인덱스:

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '역대급 bj 방송사고 터진듯?\n걸그룹 연습생 출신이라더라\n제 채널 와서 보세요..',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.',
 '제 채널에 좋은거 있어요',
 '제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤',
 '제 채널 들어와봐용 \n\n공짜(캐시/돈/깊티) 받아가세용\n피해 ㄴㄴ',
 '제 채널에 보면 큰일나는 영상있음',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ',
 '제 채널에서 꽁돈 받아가세요\U0001f973\U0001f973\U0001f973\U0001f973',
 '제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!\n절 믿고 따라오실수 있겠습니까?\n(친구가 많다면 돈을 더 벌수도 있답니다!)',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㄹㅁ',
 '제 채널에서 만원 받아가세',
 '제 채널에 네이버.웹툰 쿠키 200개 5분만에 공짜로 받는법 올려뒀어요.❤',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁㅁ',
 '제 채널에 들어오셔서 문상 2마넌 가져가세엽!!\n거짓말 아니에여 ㅠㅠ 받아가시고 말하세요ㅠ',
 '

In [307]:
spam_list

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '역대급 bj 방송사고 터진듯?\n걸그룹 연습생 출신이라더라\n제 채널 와서 보세요..',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.',
 '제 채널에 좋은거 있어요',
 '제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤',
 '제 채널 들어와봐용 \n\n공짜(캐시/돈/깊티) 받아가세용\n피해 ㄴㄴ',
 '제 채널에 보면 큰일나는 영상있음',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ',
 '제 채널에서 꽁돈 받아가세요\U0001f973\U0001f973\U0001f973\U0001f973',
 '제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!\n절 믿고 따라오실수 있겠습니까?\n(친구가 많다면 돈을 더 벌수도 있답니다!)',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㄹㅁ',
 '제 채널에서 만원 받아가세',
 '제 채널에 네이버.웹툰 쿠키 200개 5분만에 공짜로 받는법 올려뒀어요.❤',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁㅁ',
 '제 채널에 들어오셔서 문상 2마넌 가져가세엽!!\n거짓말 아니에여 ㅠㅠ 받아가시고 말하세요ㅠ',
 '

In [306]:
spam_list.extend(a)

In [308]:
spam_2 = get_KeywordDF('체널')
spam_2

['여기 답글 진짜 다 꼴보기싫네 제 체널와서 ~받아가세요~ ㅇㅈㄹ;;;;',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 체널에서 10000원 벌어가세요 ㅎㅎ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '@바두자왜 남에 체널에와서 홍보해요?     그리고 사기져?',
 '와..이거 군대휴가 나와서 체널돌리다 얼떨결에 본거같은데 그때는 와..신기하네~지금보니 아..그냥 눈물만ㅜㅜ그때나 지금이나 똑같은 영상인데 전해지는 느낌이 너무 다르다..',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '#이찬원  유튜브체널  민호님  영웅닝 희재님  #이찬원  닝축하해주시니 너무감사 해요.다들 좋은날들만 있으시길...항상 응원합니다.^^♡♡♡',
 '제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\nㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '코로나 이어려운시기  에유트부체널을통해  열심이 응원할께요     뿌잉 뿌잉    안 녕?사랑 해요~~~^^',
 '#이찬원   유튜브  체널축하합니다  회이팅 👍👍👍👍👍 트롯형님들찬원님사

In [325]:
b = del_and_prt(spam_2, 41)
b

인덱스: 0


제 체널에 간단히 꽁돈 버는법 3가지나 있어요!

진짜 있으니 해보고 말씀하세요

😍😍😍😍😍😍😍😍
--------
인덱스: 1


제 체널에서 10000원 벌어가세요 ㅎㅎ
--------
인덱스: 2


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 3


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 4


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊
--------
인덱스: 6


제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 7


제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍
--------
인덱스: 8


제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍
--------
인덱스: 9


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ
ㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 10


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 11


제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️
--------
인덱스: 12


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶
--------
인덱스: 13


제 체널에 30초면 공짜로 3마넌 

['제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 체널에서 10000원 벌어가세요 ㅎㅎ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\nㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊😲',

In [326]:
spam_list.extend(b)

In [328]:
spam_3 = get_KeywordDF('챼널')
spam_3

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오시면 아무나 5마넌 꽁짜로 얻는방법 올려놓았습니다!\n\n시간 얼마 남지 않았으니까 서둘러오세요\n\n깊카 문상 이딴거 아님 걱정 no no!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이런거 절대 절대 아님\n\n😍★😍★😍★😍★

In [329]:
spam_list.extend(spam_3)

In [331]:
spam_4 = get_KeywordDF('30초')
spam_4

['제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 챼널에 오셔서 30초만에 5마넌 가져가세요\n\n사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '30초',
 '30초전 ㄷㄷ',
 '제 채널에 문화상품권 30초만에 받는 법 있음',
 '제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ \n\n거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름.\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '@김한나 영상이 30초겠조',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고

In [338]:
d = del_and_prt(spam_4, 42)
d

인덱스: 0


제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음

궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!

기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!

정말 진짜니까 안심하고 오세용😍😍
--------
인덱스: 2


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 3


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 4


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 챼널에 오셔서 30초만에 5마넌 가져가세요

사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 6


제 채널에 문화상품권 30초만에 받는 법 있음
--------
인덱스: 7


제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ 

거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름.

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 8


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 9


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥

['제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 챼널에 오셔서 30초만에 5마넌 가져가세요\n\n사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 채널에 문화상품권 30초만에 받는 법 있음',
 '제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ \n\n거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름.\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '@김한나 영상이 30초겠조',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶

In [339]:
spam_list.extend(d)

In [340]:
spam_5 = get_KeywordDF('하루')
spam_5
### 이걸로는 무리다

['@하루1만원씩 챙겨가세요 채널클릭 빅현배 저 방송후에 팬들이 제발 먹지말라해도\n강퇴시키고 기어코 파파존스 피자에 스파게티, 콜라 먹음 ㅋㅋㅋ\n유튜브엔 안올리고 아프리카에서만 먹는다고ㅋㅋ 나참',
 '학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '@하루10,000원 받아가세요 채널클릭 ㄲㅈ 제발',
 '@하루 10,000원 받아가세요채널클릭ㅇ ㅕ8ㅗ',
 'ㅋㅋㅋ 재밋게 잘만들었네요~ㅋ 개인적인 생각은 이 어린친구에게 역사관이나 정치적인 질문은 안했으면 하네요 :) 어리기도 하고 일본에 살고있는 한국인이라.. 어떤대답을 해도 욕먹을수있는 포지션이기에ㅠ ㅠ 물론 한국인이니 한국에 대한 올바른 역사관을 갖는건 중요하지만 지금은 지금을 즐기는게 더 나아보일 나이잖아요ㅎ  고딩브이로그에서 대딩 직딩 엄마 할머니 브이로그까지 이어지길 기대할께요\nPS 영상은 하루에 한번씩 부탁드립니다ㅋㅋ 있는건 다 쓰고 나중에 본업과 유투버 사이를 고민하실때 영상업로드 주기를 결정합시다ㅋㅋ',
 '오늘도 내일도 행복한 하루 되십쇼',
 '맘도착하고이쁜쯔양~\n상처받지말고일하시길.\n전진짜먹방보면서하루스트레스날립니다 \n고마워요~♡',
 '지금 보러 와야 진정한 깡팸입니다\n어떻게 살면서 한 입으로 두 말하겠습니까\n당신은 지인들에게 난 하루에\nN깡 한다며 자랑하거나\n깡팸이라고 말하고 다녔을 것입니다\n하지만 유행이 지났다고 이렇게 버리다면\n당신은 진정한 깡팸이 아닙니다\n정말 비를 좋아하거나 깡을 좋아한다면\n꼭 1일 1깡 이상 하셔야합니다\n지인들에게 깡을 다시 전파하시고\n제 2의 깡 신드롬을 일으킵시다!!!',
 '민정님 오늘 영상 빨리빨리 진행돼서 너무 좋습니다..❤️ 몰랐던 사실 큐엔에이 영상으로 잘알고갈게요 하루하루 민정님 영상 목빠지게 기다리고 있어요...\U0001f97a\U0001f97a 오늘도 너무 이쁘시고 남은 2020년 잘보내시

In [341]:
spam_6 = get_KeywordDF('누구나')
spam_6

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '옛날에 김완선이 아주 관능 적으로 율동적으로 춤추면서 노래를  매역적으 로 불럿다 지금은 누구나 너무도 춤을잘 추는갓다👏👏👏👏👏👏👏',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '@haenseom kim 니네 가족들한테 동남아 사람 같다거나 흑인 혼혈이라고 하면 감사합니다~라고 하세요 ㅎ \n요즘 애완 동물을 반려동물이라고 칭하며 정말 가족과 같이 키우시는 분들도 많은데\n강아지를 올바르게 키우기 위해 교육시키고 환경을 조성하는 노력도 마다하지 않는 주인이란걸 조금만\n봐도 알 수 있는 상황에 진도개 혈통이란걸 누구나 아는 상황인데 잡종이라고 한다? ㅋㅋ 가정교육 제대로 받은 사람이 맞니? \n피부 조금 까맣다고 동남아 사람같다느니 짓걸여도 된다는 말이지? ㅋㅋㅋ\n내가 볼 땐 니도 정신 나간새낀거 같은데',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 

In [354]:
e = del_and_prt(spam_6, 58)
e

인덱스: 0


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ
--------
인덱스: 1


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ
--------
인덱스: 2


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오
--------
인덱스: 3


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ
--------
인덱스: 4


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 5


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 6


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍
😍
--------
인덱스: 7


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅇㅈ
--------
인덱스: 8


제 채널에 네이버웹툰 쿠키 200개 누구나 5분안에 공짜로 받는 법 올려뒀어요❤
--------
인덱스: 9


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎ

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍\n😍',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅇㅈ',
 '제 채널에 네이버웹툰 쿠키 200개 누구나 5분안에 공짜로 받는 법 올려뒀어요❤',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㄹㅇ',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!

In [355]:
spam_list.extend(e)

In [356]:
len(spam_list)

358

In [357]:
len(list(set(spam_list)))

255

In [13]:
spam_6 = get_KeywordDF('용돈')
spam_6

['٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و\n♥하루30분 광 고 보 고♥\n♥매일 용돈 1만씩 벌어가세요♥\n.\n오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ \n\nㅅㅁㄹㅁㄹㅁㄹㅁㅁㄹㄹㅁㄹㅁ',
 '🌟 다들 여기야 여기 🌟\n\n\n1. 이름 0:33\n\n2. 생년월일 0:35\n\n3. 나이 0:38\n\n4. 키/몸무게 0:42\n\n5. mbti 0:49\n\n6. 국적 0:54\n\n7. 부모님 0:56\n\n8. 형제 1:02\n\n9. 인스타 아이디 1:09\n\n10. 남자친구 + 설레는 썰 1:23\n\n11. 짝사랑 1:43\n\n12. 졸업후의 계획 1:59\n\n13. 유튜브를 시작한 계기 2:06\n\n14. 메이크업 2:19\n\n15. 이상형 2:30\n\n16. 머리 고데기 2:52\n\n17. 좋아하는 배우 아이돌 2:59\n\n18. 학교 성적 3:03\n\n19. 한국 재귀국 3:06\n\n20. 좋아하는 일본 음식 3:16\n\n21. 한국가면 하고 싶은거 3:19\n\n22. 한국이 그리울때 3:29\n\n23. 한국음식 그리워? 3:34\n\n24. 옷 구매처 3:43\n\n25. 한국에 거주할때 어디 살았는지 3:47\n\n26.꿈 생각은 어느나라 말? 3:50\n\n27. 일본에서 쓰는 이름 4:05\n\n28. 편한 언어 4:17\n\n29. 핸드폰 언어 4:26\n\n30. eju? JLPT? 4:31\n\n31. 친구들이 주로 쓰는 sns 4:41\n\n32. 지금 친한 친구랑 친해진 계기 4:52\n\n33. 일본에서 처음사귄 친구 4:58\n\n34. 혼혈친구 많은지 5:02\n\n35. 앞머리 내린 애들이 많은 이유 5:07\n\n36. 친구들 다 쌍커풀 있는 이유 5:16\n\n37. 안경쓴 친구가 없는 이유 5:33\n\n38. 남사친 5:40\n\n39. 친구들 한달 용돈 어느정도 6:08\n\n40. 친구들 키 6:18\n\n41. 친구들 한국에 대해 어떻게 생각하냐 6:22\n\n

In [30]:
f = del_and_prt(spam_6, 38)
f

인덱스: 0


٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و
♥하루30분 광 고 보 고♥
♥매일 용돈 1만씩 벌어가세요♥
.
오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ 

ㅅㅁㄹㅁㄹㅁㄹㅁㅁㄹㄹㅁㄹㅁ
--------
인덱스: 1


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥




ㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㅁ
--------
인덱스: 2


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


로롬ㄹㅁㄹㅁㄹㅁㄹㅁ
--------
인덱스: 3


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥



ㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ
--------
인덱스: 4


용돈 부족한 학생들 위해서 고생끝에 타임스프레드 버그 코드를 얻었습니다^^
제 채널 와보시면 10만원 무료입니다!
빨리 와서 얻어가세요
막히기 전에 빨리 하시고 후기들도 보세요!
찐입니다^^
--------
인덱스: 5


@꼰대희 
학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥
--------
인덱스: 6


٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و
♥하루30분 광 고 보 고♥
♥매일 용돈 1만씩 벌어가세요♥
.
오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ 

ㄹㅁㄹㅁㄹ로리리리리
--------
인덱스: 7


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ
--------
인덱스: 8


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㄹㅇㄹㅇㄹㅇ
--------
인덱스: 9


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁ
--------
인덱스: 10


٩

['٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و\n♥하루30분 광 고 보 고♥\n♥매일 용돈 1만씩 벌어가세요♥\n.\n오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ \n\nㅅㅁㄹㅁㄹㅁㄹㅁㅁㄹㄹㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n로롬ㄹㅁㄹㅁㄹㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '용돈 부족한 학생들 위해서 고생끝에 타임스프레드 버그 코드를 얻었습니다^^\n제 채널 와보시면 10만원 무료입니다!\n빨리 와서 얻어가세요\n막히기 전에 빨리 하시고 후기들도 보세요!\n찐입니다^^',
 '@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و\n♥하루30분 광 고 보 고♥\n♥매일 용돈 1만씩 벌어가세요♥\n.\n오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ \n\nㄹㅁㄹㅁㄹ로리리리리',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㄹㅇㄹㅇㄹㅇ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁ',
 '٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و\n♥하루30분 광 고 보 고♥\n♥매일 용돈 1만씩 벌어가세요♥\n.\n오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ \

In [36]:
spam_list = list(df['spams'])

In [37]:
spam_list.extend(f)

In [361]:
df = pd.DataFrame(spam_list, columns=['spams'])

In [363]:
df.to_pickle('스팸리스트_1.pkl')

In [9]:
df = pd.read_pickle('스팸리스트_1.pkl')

In [43]:
spam_7 = get_KeywordDF('꽁돈')
spam_7

['https://youtu.be/9e3xUqfkJB4 ❤️완전 쉽게 꽁돈 15000원 받는법 (네이버페이 아닙니다!) (거짓말같으면 안하셔도 되요)❤️\n\n❤️❤️틱톡 추천인 이벤트 참여하고 꽁돈 받아가세용 참여만 해도 바로 5500원 주고 바로 문상이나 기프티콘으로 교환 가능해요! 초간단 미션 달성시 총 15000원 흭득가능! 다같이 이득봅시다 ㅎㅎ😍😍',
 '제 채널에 들어오셔서 꽁돈 받아가세요\n구독도 좀 부탁드릴게욥 :)\n😍😍😍😍',
 '제 체널에 간단히 30초면 3마넌 얻는법 등\n\n꽁돈 얻는법이 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '1분만에 꽁돈 7천원 버는법 재 채널 오시면 있음!',
 '🎁크리스마스 선물🎁\n제 체널에 23000원 1분만에 버는법\n보시고 꽁돈 벌러 ㄱㄱ 25일 기준 최고금액',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '채널 클릭해서 들어오면 꽁돈 벌 수 있음.',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 받아가세요\U0001f929\U0001f929\U0001f929\U0001f929',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 15000원받는법있어요!',
 '@딸써니Dalsunny 박보람이라고 딴 댓에서도 계속 꽁돈 받으러 오세요~~ 하는 사람있어서 저렇게 말했는데 댓삭 하고 튄듯요 ㅋㅋㅋㅋ',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 말씀하세요😶',
 '님들 이거 꽁돈 벌러 오라고 하는분들 신고 한번식 만요 스팸신고 해주세요',
 '제채널 오셔서 꽁돈벌어가세여~!',
 '@COVID-19현황 알리미 그래도 5000원 꽁돈 ㅇㅈㄹ 하는 애들보단 낫다',
 '제 채널에서 꽁돈 받아가세요\U0001f9

In [40]:
spam_7[-3]

'@COVID-19현황 알리미 그래도 5000원 꽁돈 ㅇㅈㄹ 하는 애들보단 낫다'

In [45]:
g = del_and_prt(spam_7, 11)
g

인덱스: 0


https://youtu.be/9e3xUqfkJB4 ❤️완전 쉽게 꽁돈 15000원 받는법 (네이버페이 아닙니다!) (거짓말같으면 안하셔도 되요)❤️

❤️❤️틱톡 추천인 이벤트 참여하고 꽁돈 받아가세용 참여만 해도 바로 5500원 주고 바로 문상이나 기프티콘으로 교환 가능해요! 초간단 미션 달성시 총 15000원 흭득가능! 다같이 이득봅시다 ㅎㅎ😍😍
--------
인덱스: 1


제 채널에 들어오셔서 꽁돈 받아가세요
구독도 좀 부탁드릴게욥 :)
😍😍😍😍
--------
인덱스: 2


제 체널에 간단히 30초면 3마넌 얻는법 등

꽁돈 얻는법이 3가지나 있어요!

진짜 있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 3


1분만에 꽁돈 7천원 버는법 재 채널 오시면 있음!
--------
인덱스: 4


🎁크리스마스 선물🎁
제 체널에 23000원 1분만에 버는법
보시고 꽁돈 벌러 ㄱㄱ 25일 기준 최고금액
--------
인덱스: 5


제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊
--------
인덱스: 6


채널 클릭해서 들어오면 꽁돈 벌 수 있음.
--------
인덱스: 7


제 체널에 간단히 꽁돈 버는법 3가지나 있어요!
진짜 있으니 해보고 말씀하세요
😍😍😍😍😍😍😍😍
--------
인덱스: 8


제 채널에서 꽁돈 받아가세요🤩🤩🤩🤩
--------
인덱스: 9


제 체널에 간단히 꽁돈 버는법 3가지나 있어요!

진짜 있으니 해보고 말씀하세요

😍😍😍😍😍😍😍😍
--------
인덱스: 10


제 채널에서 꽁돈 15000원받는법있어요!
--------
인덱스: 11


제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 말씀하세요😶
--------
인덱스: 12


님들 이거 꽁돈 벌러 오라고 하는분들 신고 한번식 만요 스팸신고 해주세요
--------
인덱스: 13


제채널 오셔서 꽁돈벌어가세여~!
--------
인덱스: 14

['https://youtu.be/9e3xUqfkJB4 ❤️완전 쉽게 꽁돈 15000원 받는법 (네이버페이 아닙니다!) (거짓말같으면 안하셔도 되요)❤️\n\n❤️❤️틱톡 추천인 이벤트 참여하고 꽁돈 받아가세용 참여만 해도 바로 5500원 주고 바로 문상이나 기프티콘으로 교환 가능해요! 초간단 미션 달성시 총 15000원 흭득가능! 다같이 이득봅시다 ㅎㅎ😍😍',
 '제 채널에 들어오셔서 꽁돈 받아가세요\n구독도 좀 부탁드릴게욥 :)\n😍😍😍😍',
 '제 체널에 간단히 30초면 3마넌 얻는법 등\n\n꽁돈 얻는법이 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '1분만에 꽁돈 7천원 버는법 재 채널 오시면 있음!',
 '🎁크리스마스 선물🎁\n제 체널에 23000원 1분만에 버는법\n보시고 꽁돈 벌러 ㄱㄱ 25일 기준 최고금액',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '채널 클릭해서 들어오면 꽁돈 벌 수 있음.',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 받아가세요\U0001f929\U0001f929\U0001f929\U0001f929',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 15000원받는법있어요!',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 말씀하세요😶',
 '님들 이거 꽁돈 벌러 오라고 하는분들 신고 한번식 만요 스팸신고 해주세요',
 '제채널 오셔서 꽁돈벌어가세여~!',
 '제 채널에서 꽁돈 받아가세요\U0001f973\U0001f973\U0001f973\U0001f973',
 '제 채널에 오셔서 꽁돈15000원받아가세요ㅎㅎ\n1분안에15000원입니다!!  😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😀😀😀😀😀👈👈👈👈👈👈👈👈👈']

In [46]:
spam_list.extend(g)

In [47]:
spam_8 = get_KeywordDF('공짜')
spam_8

['제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '@황민아 과연 3만원을 공짜로 받을수 있을까?..',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 아님 !!',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n로롬ㄹㅁㄹㅁㄹㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㅁ',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '제 채널 들어와봐용 \n\n공짜(돈/문상/캐시) 받아가용\n피해 ㄴ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ',
 '제. 채널에 문.상 만.원 누구나 1분안에 공짜로 받는법 올려뒀어요❤.',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '@올장뚱 그 컨셉이 공짜는 아니니까, 그 돈을 한국에서 그 혐한중국작가놈에게 지불했을테니까, 차리리 중국놈들이 하던식으로 그냥 무단도용했다면 덜 기분나빴을까나...',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊

In [57]:
h = del_and_prt(spam_8, 53)
h

인덱스: 0


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!
!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 1


제 채널 들어와봐용 

공짜(돈/캐시/깊티) 받아가세요 !
사기 ㄴㄴ
--------
인덱스: 2


제 채널 들어와봐용 

공짜(돈/캐시/깊티) 받아가세요 !
사기 아님 !!
--------
인덱스: 3


제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!

정말 진짜니까 안심하고 오세용😍😍
--------
인덱스: 4


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


로롬ㄹㅁㄹㅁㄹㅁㄹㅁ
--------
인덱스: 5


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥



ㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ
--------
인덱스: 6


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥




ㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㅁ
--------
인덱스: 7


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 8


@꼰대희 
학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥
--------
인덱스: 9


제 채널 들어와봐용 

공짜(돈/문상/캐시) 받아가용
피해 ㄴ
--------
인덱스: 10


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ
--------
인덱스: 11


제. 채널에 문.상 만.원 누구나 1분안에 공짜로 받는법 올려뒀어요❤.
--------
인덱스: 12


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 13


제 채널 들어와봐

['제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 아님 !!',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n로롬ㄹㅁㄹㅁㄹㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㅁ',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '제 채널 들어와봐용 \n\n공짜(돈/문상/캐시) 받아가용\n피해 ㄴ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ',
 '제. 채널에 문.상 만.원 누구나 1분안에 공짜로 받는법 올려뒀어요❤.',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져

In [59]:
spam_list.extend(h)

In [60]:
spam_9 = get_KeywordDF('문상')

인덱스: 0


제 챼널오셔서 30초만에 5마넌 가져가세요!

이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!
--------
인덱스: 2


이야 유튜브 어쩌다 이렇게됬냐 한 뎃글에 낚시빌런, 광고빌런, 문상빌런, 대깨문 다있냐 그것도 몇명씩이나
--------
인덱스: 3


그래도 문상년놈들보단 낫지
--------
인덱스: 4


@박진우 차라리 저런게 낫지 ㅋㅋ
??:제 채널에오셔서 공짜 문상받아가세요!
이거보단 좋은의미잖어 ㅋㅋ
--------
인덱스: 5


제 챼널오셔서 30초만에 5마넌 가져가세요!

이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 6


제 채널에 문상 5천원 꽁으로 받는법 있어요!!
--------
인덱스: 7


🤗제 영상에 문상 5000원 올려놨어여!🤗


필요하신분 있으면 공짜로 받아가셔용0
--------
인덱스: 8


https://youtu.be/ElMIg941zVs
↑댓글만 달면 문상 2만원이 무료?!
--------
인덱스: 9


🤗제 영상에 문상 5000원올려놨어여!🤗


필요하신분 있으면 공짜로 받아가셔용^0
--------
인덱스: 10


🤗제 영상에 문상 5000원 올려놨어여!🤗


필요하신분 있으면 공짜로 받아가셔용^0
--------
인덱스: 11


수입 들어오면 깊카나 문상으로 돈뿌립니다 구독 부탁드려요
--------
인덱스: 12


제 챼널오시면 아무나 5마넌 꽁짜로 얻는방법 올려놓았습니다!

시간 얼마 남지 않았으니까 서둘러오세요

깊카 문상 이딴거 아님 걱정 no no!

😍★😍★😍★😍★😍★😍★😍★😍★😍★

In [64]:
a = del_and_prt(spam_9, 17)
a

인덱스: 0


제 채널에 문상 5천원 꽁으로 받는법 있어요!!
--------
인덱스: 1


제 채널에 쉽게 문상 얻는법 올려둠
--------
인덱스: 2


수입 들어오면 깊카나 문상으로 돈뿌립니다 구독 부탁드려요
--------
인덱스: 3


제 채널 들어와봐용 

공짜(돈/문상/캐시) 받아가용
피해 ㄴ
--------
인덱스: 4


제 영상에서 문상 받아가세요 ㅎㅎ
--------
인덱스: 5


제 챼널오시면 누구든지 5마넌 꽁짜로 얻는방법 올려놓음

시간 얼마 안남았으니까 서둘러오세요

깊카 문상 이딴거랑 다르니까 믿고 오세요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 6


이런말 쓰기 뭐하지만
마음이 급해 써봅니다
저는 하나뿐인동생에게 유튜브 구독자 100명계정을 생일선물로 주기로 했습니다
앞으로 5개월정도 남았습니다
저는 제가 아끼는 동생이 실망하는걸 보기싫습니다 만약에 제가 구독자 100명이된다면 제가 가지고 있는 3만원을 문상 5000원으로 바꾸어서 추후 6명에게 드리겠습니다

추신:강아지는.사랑입니다 저도 포메를 키우는데요 말은안듣지만 귀여워서 앙 하고 물어뜯어버리고싶네요
--------
인덱스: 7


제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!
--------
인덱스: 8


제 채널에 드러와서 문상 받아가세요
신입 유튜번데 구독도 좀 부탁드릴게요
🧡🧡🧡
--------
인덱스: 9


제 영상에서 문상 받아가세요 ㅎㅎ
유튜버 지망생입니다. 제발 구독 눌러주세요.
🥲🥲😍😍😍😍😍
--------
인덱스: 10


제 채널에서 깊카, 문상 무료로 뿌려요 받아가세요!!
--------
인덱스: 11


유튜버 지망생입니다.
제발 구독 눌러주세요 ㅠㅠ 
참고로 문상도 받으실수 있어요
--------
인덱스: 12


🤗제 영상에 문상 5000원 올려놨어여!🤗


필요하신분 있으면 공짜로 받아가셔용❤ㅓ
--------
인덱스: 13


제 챼널오시면 누구든지 5마넌 꽁짜

['제 채널에 문상 5천원 꽁으로 받는법 있어요!!',
 '제 채널에 쉽게 문상 얻는법 올려둠',
 '수입 들어오면 깊카나 문상으로 돈뿌립니다 구독 부탁드려요',
 '제 채널 들어와봐용 \n\n공짜(돈/문상/캐시) 받아가용\n피해 ㄴ',
 '제 영상에서 문상 받아가세요 ㅎㅎ',
 '제 챼널오시면 누구든지 5마넌 꽁짜로 얻는방법 올려놓음\n\n시간 얼마 안남았으니까 서둘러오세요\n\n깊카 문상 이딴거랑 다르니까 믿고 오세요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '이런말 쓰기 뭐하지만\n마음이 급해 써봅니다\n저는 하나뿐인동생에게 유튜브 구독자 100명계정을 생일선물로 주기로 했습니다\n앞으로 5개월정도 남았습니다\n저는 제가 아끼는 동생이 실망하는걸 보기싫습니다 만약에 제가 구독자 100명이된다면 제가 가지고 있는 3만원을 문상 5000원으로 바꾸어서 추후 6명에게 드리겠습니다\n\n추신:강아지는.사랑입니다 저도 포메를 키우는데요 말은안듣지만 귀여워서 앙 하고 물어뜯어버리고싶네요',
 '제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!',
 '제 채널에 드러와서 문상 받아가세요\n신입 유튜번데 구독도 좀 부탁드릴게요\n\U0001f9e1\U0001f9e1\U0001f9e1',
 '제 영상에서 문상 받아가세요 ㅎㅎ\n유튜버 지망생입니다. 제발 구독 눌러주세요.\n\U0001f972\U0001f972😍😍😍😍😍',
 '제 채널에서 깊카, 문상 무료로 뿌려요 받아가세요!!',
 '유튜버 지망생입니다.\n제발 구독 눌러주세요 ㅠㅠ \n참고로 문상도 받으실수 있어요',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용❤ㅓ',
 '제 챼널오시면 누구든지 5마넌 꽁짜로 얻는방법 올려놓음\n\n시간 얼마 안남았으니까 서둘러오세요\n\n깊카 문상 이딴거랑 다르니까 믿고 오세요\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신

In [65]:
spam_list.extend(a)

In [66]:
spam_10 = get_KeywordDF('꽁으로')
spam_10

인덱스: 0


제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!
--------
인덱스: 1


제 채널에 문상 5천원 꽁으로 받는법 있어요!!
--------
인덱스: 2


🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥제 체널에 10투자하면  20만원 꽁으로 받을수있어요🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
--------


['🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥제 체널에 10투자하면  20만원 꽁으로 받을수있어요🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥',
 '제 채널에 문상 5천원 꽁으로 받는법 있어요!!',
 '제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!']

In [67]:
spam_list.extend(spam_10)

In [68]:
spam_11 = get_KeywordDF('만원')
spam_11

인덱스: 0


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


로롬ㄹㅁㄹㅁㄹㅁㄹㅁ
--------
인덱스: 1


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😮😦
--------
인덱스: 2


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㄹㄹㅁㅁㄹㅁㅁㄹ
--------
인덱스: 3


돈없는 학생 분들을 위해 준비했습니다!
제 채널 오시고 10만원 무료로  얻어가세요
--------
인덱스: 4


제 채널에 오셔서 2만원 받아가세요~
2일 남았어요!! 서두르세요 ㄹㅇ줍니다
(12월 31일까지)
--------
인덱스: 5


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ
ㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 6


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥
--------
인덱스: 7


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 8


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶😶
--------
인덱스: 9


@김한나 이 분한테 5만원 사기 당함
--------
인덱스: 10


@mmm제채널 하루1만원 받아가세요mmm 와 이거 진짜네 ㄷㄷ 이걸로 강남에 120억짜리 빌딩삼 ㄱㅅㄱㅅ
--------
인덱스: 11


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊
--------
인덱스: 12


🥳제 프로필 오셔서 ㅁㅅ.ㄱㅋ 5만원권 바로 받아가세요╥﹏╥。◕‿◕ 。(❛◡❛)✿ᕙ

['11:07 솔직히 5만원 짜리 70장 계산한 사람 모여라 ㅋㅋㅋ',
 '제 챼널에 5분만에 만원받는 법 있습니다. 진짜루요!!',
 '\U0001f973제 프로필 오셔서 ㅁㅅ,ㄱㅋ 5만원권 바로 받아가세요。◕‿◕ 。╥﹏╥。◕‿◕ 。╥﹏',
 '@거지준영  200만원 입금 했습니다',
 '양팡 복귀함 ㄹㅇ 구라면 100만원 지른 롤계정 삭제https://youtu.be/4lyLu91rjx4',
 '@10분만에 만원을 버는 간단한 방법 진짜 유행이냐...ㅉㅉ',
 '궁금해 하지말고 검색해\n\n"인싸잡"\n나이/지역/장소 X\n 합법적인 간단알바\n 하루 3만원\n카카오톡 상단 돋보기 클릭!!\n""인싸잡""검색',
 '5만원만 저 주시면 안되나요...',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ',
 '@글레이시아 케이크 하나가 3만원 정도 하네요.. 그래도 영상에서 보인 정성을 생각하면 적당하다고도 느껴지네요',
 '@진진돌이 5천만원이라는 언급에서 보아 생각보다는 어린분이신거 같으니 치료효과가 빨리 나타날겁니다. 혹여 병원에서 약 복용을 같이 권하면 처음에는 복용없이 하겠다고 말하시고, 차도를 지켜보시길 바랍니다.',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㄹㅇㄹㅇㄹㅇ',
 '이런말 쓰기 뭐하지만\n마음이 급해 써봅니다\n저는 하나뿐인동생에게 유튜브 구독자 100명계정을 생일선물로 주기로 했습니다\n앞으로 5개월정도 남았습니다\n저는 제가 아끼는 동생이 실망하는걸 보기싫습니다 만약에 제가 구독자 100명이된다면 제가 가지고 있는 3만원을 문상 5000원으로 바꾸어서 추후 6명에게 드리겠습니다\n\n추신:강아지는.사랑입니다 저도 포메를 키우는데요 말은안듣지만 귀여워서 앙 하고 물어뜯어버리고싶네요',
 '@10분만에 만원을 버는 간단한 방법 시빵ㅏㄱ',
 '이걸 아직도

In [133]:
a = del_and_prt(spam_11, 43)

인덱스: 0


11:07 솔직히 5만원 짜리 70장 계산한 사람 모여라 ㅋㅋㅋ
--------
인덱스: 1


제 챼널에 5분만에 만원받는 법 있습니다. 진짜루요!!
--------
인덱스: 2


🥳제 프로필 오셔서 ㅁㅅ,ㄱㅋ 5만원권 바로 받아가세요。◕‿◕ 。╥﹏╥。◕‿◕ 。╥﹏
--------
인덱스: 3


양팡 복귀함 ㄹㅇ 구라면 100만원 지른 롤계정 삭제https://youtu.be/4lyLu91rjx4
--------
인덱스: 4


궁금해 하지말고 검색해

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 5


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㄹㄹㅁㄹㅇㅇㄹㄹㅇ
--------
인덱스: 6


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㄹㅇㄹㅇㄹㅇ
--------
인덱스: 7


이런말 쓰기 뭐하지만
마음이 급해 써봅니다
저는 하나뿐인동생에게 유튜브 구독자 100명계정을 생일선물로 주기로 했습니다
앞으로 5개월정도 남았습니다
저는 제가 아끼는 동생이 실망하는걸 보기싫습니다 만약에 제가 구독자 100명이된다면 제가 가지고 있는 3만원을 문상 5000원으로 바꾸어서 추후 6명에게 드리겠습니다

추신:강아지는.사랑입니다 저도 포메를 키우는데요 말은안듣지만 귀여워서 앙 하고 물어뜯어버리고싶네요
--------
인덱스: 8


이걸 아직도 몰라?

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 9


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㄹㅁ
--------
인덱스: 10


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀

In [ ]:
'ㅁㅅ', 'ㄱㅋ'

In [134]:
spam_12 = get_KeywordDF('마넌')
spam_12

인덱스: 0


제 챼널오면 누구나 5마넌 꽁짜로 얻는방법 있어요~

선착순이니까 서두르세용~(12월31일까지 진행중) 

기프트캬드, 문화샹품권 이런거 아닙니다. 믿고오세용

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~

정말 진짜니까 안심하고 오세용😍😍
--------
인덱스: 2


제 챼널오셔서 30초만에 5마넌 가져가세요!

이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 3


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 4


제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~

거짓말 아니니까 믿고 오세요😍😍
--------
인덱스: 5


제 챼널에 아무나 가입 5마넌 얻는법 올려놨음
진짜 거짓말 절대로 아니니니까 믿고오세요
기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요
😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 6


제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음

궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!

기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 7


제 챼널오면 5마넌 얻는법 올려놓음. 

거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용

기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 8


제 챼널오셔서 30초만에 5

['제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤하ㅏ핳ㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '@??? 제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤???',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅎㅇㅎㅇ',
 '제 챼널오시면 누구든지 5마넌 꽁짜로 얻는방법 올려놓음\n\n시간 얼마 안남았으니까 서둘러오세요\n\n깊카 문상 이딴거랑 다르니까 믿고 오세요\n\n😍★😍★😍★😍★😍

In [135]:
a = del_and_prt(spam_12, 67)

인덱스: 0


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 1


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤하ㅏ핳ㅎ
--------
인덱스: 2


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ
--------
인덱스: 3


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!
!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 4


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄
--------
인덱스: 5


제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!

정말 진짜니까 안심하고 오세용😍😍
--------
인덱스: 6


제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️
--------
인덱스: 7


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 8


@??? 제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤???
--------
인덱스: 9


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까

In [136]:
spam_list.extend(a)

In [137]:
spam_13 = get_KeywordDF('거짓말')

인덱스: 0


제 챼널오셔서 30초만에 5마넌 가져가세요!

이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 2


제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~

거짓말 아니니까 믿고 오세요😍😍
--------
인덱스: 3


저가 다정한 부부 대해서 너무 화가 나서 용서  못할것 같고 사실 대로  말해 야할것 같아서 영상  만들도  이글  놀여요 영상 도 많 들었어요 목슴 걸고 링크  누러서 꼭봐주세요
https://youtu.be/3eeMxhoC0GY
36살 차이 유튜버 다정한 부부의 진실?'
목슴걸고 한번씩봐주세요  홍보 도해주세요 저가  청화대  계시판에  올렸어요
 청원동이  랑  이링크  공유 많이해주세요 부탁합니다  이분들  처벌  받아야합니다
100명정도가  동참해야합니다 그래야지  처벌 받을수있어요
다정한부부이야기영상만들었어요링크 누르고봐주세요 https://www1.president.go.kr/petitions/Temp/bQ65qY

저가  외 영상 을 만들었냐면요 돈 벌고 싶어서만든것도 아니고 사람들한대  진실을 알리고싶어서 영상만들게되었습니다   지인 친구들 가족 들한대 공유 많이해주세요  저는  해외 사는  사람들한대도  알리고싶을 정도록 화가나는  다정한부부   감옥  갈수있게 할것입니다   여러분 도와주세요 
각종 의혹에 휩싸인 유튜버 ‘다정한 부부’가 의혹을 부인했다. 

18일 방송된 SBS 시사교양 프로그램 ‘궁금한 이야기 Y’에는 38살 신랑과 74살 아내, 36세 차로 화제를 모은

In [158]:
a = del_and_prt(spam_13, 41)

인덱스: 0


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 1


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤하ㅏ핳ㅎ
--------
인덱스: 2


@??? 제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤???
--------
인덱스: 3


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅎㅇㅎㅇ
--------
인덱스: 4


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅋㅋㅋ♡♡
--------
인덱스: 5


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅋㅎㅋㅎㅋㅎㅋ
--------
인덱스: 6


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ
--------
인덱스: 7


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이

In [159]:
spam_list.extend(a)

In [160]:
spam_14 = get_KeywordDF('프로필')

인덱스: 0


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있습니다👍
--------
인덱스: 1


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍
😍
--------
인덱스: 2


🥳제 프로필 오셔서 ㅁㅅ.ㄱㅋ 5만원권 바로 받아가세요╥﹏╥。◕‿◕ 。(❛◡❛)✿ᕙ(•̀‸•́‶)ᕗ
--------
인덱스: 3


🥳제 프로필 오셔서 ㅁㅅ,ㄱㅋ 5만원권 바로 받아가세요。◕‿◕ 。╥﹏╥。◕‿◕ 。╥﹏
--------
인덱스: 4


유트브 프로필이랑 이분이랑 똑같은...분?
--------
인덱스: 5


제프로필 이랑 똫갔네여
--------
인덱스: 6


@배민귀엽네 프로필 어몽어스하고 잼민이
--------
인덱스: 7


#이찬원유튜브 프로필사진을 찬원님 실물사진으로 바꿔주세요~ 솟대배경은 좋은데 사진이 너무 유아틱하고 만화적입니다. 동그라미 안에만 이찬원 실물사진을 넣어주시면 좋겠어요. 대문 꾸며주신분께 감사드립니다^^
--------
인덱스: 8


#이찬원_유튜브_프로필사진은 이찬원 사진으로 올려주면 더 좋을것같아요 대문사진도 프로필 사진도 귀엽긴한데 너무 유아용같은 느낌이 살짝 드네요
--------
인덱스: 9


#이찬원 늘 응원하고 사랑합니다^^ 대문 예쁘게 꾸며주신 분께 감사드립니다. 배경은 좋은데 동그라미 안에는 찬원님 실물사진 넣어주세요~^^  너무 아기같은 느낌이라서 프로필 실물사진으로 바꿔주세요. 부탁드려요~~
--------
인덱스: 10


유튜브프로필사진넘 귀여워요
30만가자~고고고
--------
인덱스: 11


#이찬원 님 영상 꼬마웡,,
형님들 응원감사해요.
배너랑 프로필도 너무 귀여워용
--------
인덱스: 12


찬또배기 유튜브 프로필 넘 귀여워졌네요ㅋㅋㅋ 그 공약들 꼭 지키시길...!😁😁
--------
인덱스: 13


찬원오빠 프로필 사진 바꾸셨네용~ 넘넘 귀여워요 ㅎㅎ 글구 영상속에서 이찬원TV개국했다고 하니까 "와아아아~"라고 말하는 희재?오빠도 

In [172]:
a = del_and_prt(spam_14, 4)

인덱스: 0


🥳제 프로필 오셔서 ㅁㅅ,ㄱㅋ 5만원권 바로 받아가세요。◕‿◕ 。╥﹏╥。◕‿◕ 。╥﹏
--------
인덱스: 1


유튜브프로필사진넘 귀여워요
30만가자~고고고
--------
인덱스: 2


🥳제 프로필 오셔서 ㅁㅅ.ㄱㅋ 5만원권 바로 받아가세요╥﹏╥。◕‿◕ 。(❛◡❛)✿ᕙ(•̀‸•́‶)ᕗ
--------
인덱스: 3


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있습니다👍
--------
인덱스: 4


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍
😍
--------


In [173]:
spam_list.extend(a)

In [175]:
spam_15 = get_KeywordDF('문화')

인덱스: 0


제 챼널오면 누구나 5마넌 꽁짜로 얻는방법 있어요~

선착순이니까 서두르세용~(12월31일까지 진행중) 

기프트캬드, 문화샹품권 이런거 아닙니다. 믿고오세용

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 2


원작 작가의 화친공주에서 빵쯔라는 단어가 수도 없이 많이 나오고, 한국 문화를 비하하는데 자존심도 안상하냐?
--------
인덱스: 3


무식한 건 감출 수가 없네
니같은 것들은 사회의 발전에 하등 도움이 안된다. 비판하진 못해도 경각심을 갖고 목소리를 내야지 그렇게 하지도 못하면서 스스로 까내리는 꼴이란 ㅋㅋ 중국한테 문화적으로 다 먹히고 나서야 남 탓하겠지 니 같은 것들 탓인데. 재미로 드라마보는 거면 보시돼, 이게 왜 문제인지는 아셔야죠. 되려 사람들을 유난스럽게 만드는 행동은 당신의 이전 행보를 짐작게 하네요
--------
인덱스: 4


제 챼널에 아무나 가입 5마넌 얻는법 올려놨음
진짜 거짓말 절대로 아니니니까 믿고오세요
기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요
😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 5


제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음

궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!

기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 6


청소년만 그런게 아니라 20대도 똑같음 그리고 문해력이랑 상관없이 웃어른 존중하는 문화가 사라져가는 게 가장 큰 이유. 아무리 븅신같이 살았어도 나보다 인생 오래 살았고 그만큼 배운 게 있을테니까 우선 귀담아들으면 그 중에서 건질게 분명 있는데 그냥 꼰, 틀 

In [252]:
a = del_and_prt(spam_15, 18)

인덱스: 0


제 챼널에 누구나 1분에 2마넌 얻는법 올려놨습니다~~

사기 정말로 아니니까 믿고 오세요~~(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 1


제 챼널오면 5마넌 얻는법 올려놓음. 

거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용

기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 2


제 챼널에 아무나 가입 5마넌 얻는법 올려놨음
진짜 거짓말 절대로 아니니니까 믿고오세요
기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요
😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 3


제 챼널오시면 누구든지 2마넌 꽁짜로 얻는방법 올려놨습니다

기간 얼마 안남았으니까 서두르세용~(12월31일까지 진행중) 

기프트캬드, 문화샹품권 이딴거 저어얼대 아닙니다.

😍❤😍❤😍❤😍❤😍❤😍❤😍❤😍❤😍❤😍❤
--------
인덱스: 4


제 채널에 문화상품권 30초만에 받는 법 있음
--------
인덱스: 5


❤️❤️ 대댓광고 사기 알려준다 ❤️❤️
👍네이버페이
통장 없이도 문화상품권으로 지급받을 수 있음❤️

👎캐시몽
하루에 100포인트로 10일 모아야 천원 남짓ㅠㅠ
되게 별로고 병신되는 기분임😡😣
--------
인덱스: 6


제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음

궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!

기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 7


제 챼널에 오셔서 30초만에 5마넌 가져가세요

사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--

In [253]:
spam_list.extend(a)

In [255]:
spam_16 = get_KeywordDF('3마')

인덱스: 0


제 체널에 간단히 30초면 3마넌 얻는법 등

꽁돈 얻는법이 3가지나 있어요!

진짜 있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 1


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 2


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 3


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 4


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!
진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 6


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!
!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 7


제 체널에 누구나 쉽게 30초면 3마넌 공짜로  얻는법 있어요!
진짜있으니 해보고 말씀하세요!!😭
--------
인덱스: 8


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 9


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😮
--------
인덱스: 10


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 11


제 체널에 30초면 누구나 3마넌 얻는법 있어요!! 진짜있으니 해보고 말씀하세요😭😭
--------


In [256]:
spam_list.extend(spam_16)

In [257]:
spam_17 = get_KeywordDF('3만')

인덱스: 0


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😮😦
--------
인덱스: 1


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶😶
--------
인덱스: 2


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊
--------
인덱스: 3


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊😲
--------
인덱스: 4


@네이버페이로5분만에 진짜 만원벌기 밑에 3만원보다 딸리는데 노력하십시오
--------
인덱스: 5


뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅
--------
인덱스: 6


@박보람  3만 원밖에 안 줘요?
--------
인덱스: 7


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶
--------
인덱스: 8


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 9


으아 3만 남았다 !!!!
--------
인덱스: 10


우와 파뿌리 3시간 지났는데 조회수가23만이다ㄷㄷ그리고 파뿌리 화이팅❤️❤️
--------
인덱스: 11


"꿀알바"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원

카카오톡 상단 돋보기 클릭!!
""m꿀잡""검색
--------
인덱스: 12


11:07 노랭이가 영혼까지 끌어모아서 70장 모았다
진렬이는 시작할때 애국가 부르며 촤라락 넘기고
10초.뒤에 70장이라고 하면 3만5천원 버는데
--------
인덱스: 13


궁금해 하지말고 검색해

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색


In [277]:
a = del_and_prt(spam_17, 19)

인덱스: 0


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊
--------
인덱스: 1


"인싸잡"

나이/지역/장소 X

 합법적인 간단알바

 하루 3만원

카카오톡 상단 돋보기 클릭!!

""인싸잡""검색
--------
인덱스: 2


궁금해 하지말고 검색해

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 3


"인싸알바"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원

카카오톡 상단 돋보기 클릭!!
""go인싸""검색
--------
인덱스: 4


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😮😦
--------
인덱스: 5


"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 6


이걸 아직도 몰라?

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 7


재미있다~

"인싸잡"
나이/지역/장소 X
 합법적인 간단알바
 하루 3만원
카카오톡 상단 돋보기 클릭!!
""인싸잡""검색
--------
인덱스: 8


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶
--------
인덱스: 9


"꿀알바"

나이/지역/장소 X

합법적인 간단알바

하루 3만원



카카오톡 상단 돋보기 클릭!!

""m꿀잡""검색
--------
인덱스: 10


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 11


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊😲

In [278]:
spam_list.extend(a)

In [279]:
spam_18 = get_KeywordDF('영상')
#이건 못쓰겠따

인덱스: 0


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


로롬ㄹㅁㄹㅁㄹㅁㄹㅁ
--------
인덱스: 1


이렇게 길게 올려주셔서 감사합니다. 캐나다에서 보니라 몇가지 짧은 영상밖에 못봤는데, 이렇게 한번에 보니 넘 좋네요.. 계속 올려주세요. 부탁합니다.
--------
인덱스: 2


인기동영상 싹쓸이하는거봐❤️
--------
인덱스: 3


인기급상승동영상1 대박!~
--------
인덱스: 4


역사왜곡 혐한원작 드라마가 인기동영상 1위인거 상당히 거슬리네요
--------
인덱스: 5


우리나라에선 고소하다는 맛 표현을 많이 사용하겠네요. 유용한 표현 알려주시셔서 감사합니다. 그리고 오랜만에 영상 올려주시셔서 또 다시 감사합니다.
--------
인덱스: 6


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㄹㄹㅁㅁㄹㅁㅁㄹ
--------
인덱스: 7


알고리즘이 날 인도했지만 본 유튭 영상중 가장 충격적이었다.... 고소하다 라는 말에 별생각 없이 지냈는데 생각해보니 크리미와 너티가 다른거였지..
--------
인덱스: 8


ㅠㅠ 최근에 스토리에 떠서 구독했는데 최근 영상이 1년 전 영상이라.. 근데 이렇게 영상 올려주시다니 너무 좋아요ㅠㅠ 사랑해요
--------
인덱스: 9


영상 자주 올려주세요~^^
이 방송에서 나왔던 tight란 포현 요즘 일할 때 잘 써먹고 있습니다ㅋ
--------
인덱스: 10


와 그저께 구독했는데 1년만에 올라오는 영상이라니.. 다음영상 기대합니다!!
--------
인덱스: 11


이 언니 영상 왤케 중독됨....
--------
인덱스: 12


선생님 영상 너무 잘 보고있습니다 ! 영상이 안올라와서 아쉬웠는데 올라와서 너무 좋아요 자주 올려주세요🥰
--------
인덱스: 13


진짜 너무 좋은데 영상 업데이트 없어서 구취해야되니 고민했는데 올려주셔서 감사합니다~!
--

영상을 빨리 내 놓으히오!
--------
인덱스: 1016


36살 차이 유튜버 다정한 부부의 진실?'
목슴걸고 한번씩봐주세요  홍보 도해주세요 저가  청화대  계시판에  올렸어요
 
영상 도 많 들었어요 목슴 걸고 링크  누러서 꼭봐주세요
https://youtu.be/3eeMxhoC0GY
청원동이  랑  이링크  공유 많이해주세요 부탁합니다  이분들  처벌  받아야합니다
100명정도가  동참해야합니다 그래야지  처벌 받을수있어요
https://www1.president.go.kr/petitions/Temp/bQ65qY

저가  외 영상 을 만들었냐면요 돈 벌고 싶어서만든것도 아니고 사람들한대  진실을 알리고싶어서 영상만들게되었습니다   지인 친구들 가족 들한대 공유 많이해주세요  저는  해외 사는  사람들한대도  알리고싶을 정도록 화가나는  다정한부부   감옥  갈수있게 할것입니다   여러분 도와주세요 
각종 의혹에 휩싸인 유튜버 ‘다정한 부부’가 의혹을 부인했다. 

18일 방송된 SBS 시사교양 프로그램 ‘궁금한 이야기 Y’에는 38살 신랑과 74살 아내, 36세 차로 화제를 모은 유튜버 다정한 부부가 등장했다.

앞서 다정한 부부는 36세 나이 차, 소박한 먹방으로 주목받으며 어설프지만 순박한 일상 모습이 담긴 영상을 통해 많은 이들의 사랑을 받았다. 특히 100명 남짓이던 구독자를 순식간에 쓸어모으며 단숨에 인기 유튜브 채널에 등극하기도 했다.
그러나 부부는 영상에서 나이 차이를 실제보다 적게 속였고, 혼인신고도 하지 않았다는 사실이 알려지며 대중들의 의심을 받기 시작했다. 특히 한 네티즌은 댓글로 다정한 부부의 아내가 과거 ‘티켓다방’(불법 성매매업소)을 운영했다고 말했고, 그가 미성년자를 고용한 적이 있 (부부가) 서로를 이모, 삼촌이라고 부른다는 주장을 해 논란이 됐다.

이날 방송에서 한 제보자는 “(다정한 부부) 아내는 성매매를 강요하지 않았을 뿐”이라며 “다방에서 일수로 해서 돈을 안 갚은 곳이 많았다. 그 때문에 도망갔다”고 폭로해 

In [280]:
spam_19 = get_KeywordDF('이벤트')

인덱스: 0


@이벤트저장소 싫어요!!!!!!!😍☆😍☆
--------
인덱스: 1


@이벤트저장소 님이나 많이 가지셈
--------
인덱스: 2


@이벤트저장소 진짜 민폐..
--------
인덱스: 3


@이벤트저장소 그만좀 하세요 화나게 하지 말고요 그딴거에 관심 없어요
--------
인덱스: 4


제 챼널오면 5마넌 얻는법 올려놓음. 

거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용

기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 5


@이벤트저장소 개소리임
--------
인덱스: 6


혹시 이벤트로 먹방같은거 찍어주실 생각없나요? ;;;
--------
인덱스: 7


@이벤트저장소 5만원획득=이상한 듣보 사이트 5만포인트를 칭함.
결국 추천인 누르면 지한테만 유리한 수법임.
괜히 헛수고하는 호구 업기를
+이런 ㅈ같은 광고하는 사기꾼들 댓글 옆에 동그라미3개 클릭후 스팸 및 상업적 광고 신고 ㄱㄱㄱㄱ
--------
인덱스: 8


사이다 리뷰이벤트 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
--------
인덱스: 9


민정님 이제 5만명 이벤트 준비하셔야죠 😝
--------
인덱스: 10


딱히 좋아하지도 싫어하지도 않았던거 같음.
한국사람 하도 많이 오고, 확실히 한국이랑 일본은 식당 에티켓? 개념이 반대 되는 경우가 많음.
그런거 모르고 식당을 가면, 외국인인거 알면서도 그냥 일본어로 우다다 말하다가, 아 일본어 모르나보네 하고 무시해버리는 경우 종종있음.
내생각엔 일본은 다른것보다  '디테일'이란것에 진절머리가 날정도로 엄청나게 집착하는 민족 같음.
물건이든 식당이든 문화든 예절이든...

신입사원되면  '명함주고받는 예절'을 연습해야 할정도의 절차가 있다는것도 충격

그리고 정말작은 지방소도시의 자치단체의 축제나 전통행사 또는 이벤트의 퀄리티를 보면 얼마나 부강한 나라인지도 깨닫게됨.
--------
인덱스: 11


제 챼널오면 

In [318]:
a = del_and_prt(spam_19, 33)

인덱스: 0


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ
--------
인덱스: 1


@이벤트저장소 제 채널에 엄청난 보물이 있어요!!
--------
인덱스: 2


@이벤트저장소 싫어요!!!!!!!😍☆😍☆
--------
인덱스: 3


@??? 제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤???
--------
인덱스: 4


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅎㅇㅎㅇ
--------
인덱스: 5


제 채널에 합 4만원받을수 있는 이벤트 올려놨어요~      1
--------
인덱스: 6


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅋㅋㅋ♡♡
--------
인덱스: 7


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅋㅎㅋㅎㅋㅎㅋ
--------
인덱스: 8


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ
--------
인덱스: 9


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에

In [319]:
spam_list.extend(a)

# 스팸 추출

In [322]:
spam_list = list(set(spam_list))

In [324]:
df = pd.DataFrame(spam_list, columns = ['spams'])

In [325]:
df.to_pickle('스팸추출.pkl')

### 유튜브 리뷰로 학습
- 이진분류 모델을 해야한다. 이 때 스팸이 아닌 댓글들을 0으로 라벨링을 하고, 스팸인 걸 1로 라벨링을 해야한다.  
- 대신 0으로 라벨링을 하기 위해서는 스팸이 아닌 댓글들만을 뽑아야 한다.

# 이모티콘
- 포함한 경우와 포함하지 않은 경우
- 딥러닝 vs 머신러닝 (표본을 많이 찾을수가 없다)